# Data Cleaning on Financial Market Data
Data is gathered from Bank of England and Office for National Statistics.

In [1]:
# import all files
import pandas as pd


#this code gets the certain sheet we want to upload.
gdp = pd.read_excel('gdp.xlsx')
cpih = pd.read_excel('CPIHAnnualRate.xlsx')
bank_rate = pd.read_excel('BankRate.xlsx')


# Data Cleaning GDP
Since the data here is by quarter, we need to change it to year, by calculating the average of each year.

In [2]:
# rename columns
gdp = gdp.rename(columns={'gdp': 'rate'})
gdp.head()

,YEAR,GDP
0,1955 Q2,0.0
1,1955 Q3,2.0
2,1955 Q4,-0.5
3,1956 Q1,1.1
4,1956 Q2,-0.1


In [3]:
# lowercase all
gdp.columns = [column.lower() for column in gdp.columns]


# separate Quarter into new column
gdp['quarter'] = gdp['year'].str[-3:]
# remove Quarter from year
gdp['year'] = gdp['year'].str[:-3]
gdp.head()

,year,gdp,quarter
0,1955,0.0,Q2
1,1955,2.0,Q3
2,1955,-0.5,Q4
3,1956,1.1,Q1
4,1956,-0.1,Q2


In [4]:
gdp = gdp.rename(columns={'gdp': 'rate'})

gdp.dtypes

year        object
rate       float64
quarter     object
dtype: object

In [5]:
# change year to integer
gdp['year']=gdp['year'].astype(int)
gdp.dtypes

year         int64
rate       float64
quarter     object
dtype: object

In [6]:
# filter data from 2014 only
gdp=gdp[(gdp['year']>2013) & (gdp['year']<2023)]
# sum data by year.
gdp_filt = gdp.groupby('year')['rate'].mean().reset_index()
gdp_filt.head()

,year,rate
0,2014,0.800
1,2015,0.475
2,2016,0.500
3,2017,0.675
4,2018,0.175


# Data Cleaning CPIH Data

In [7]:
cpih = cpih.rename(columns={'cpih': 'rate'})
cpih.head()

,YEAR,CPIH
0,1989,5.7
1,1990,8.0
2,1991,7.5
3,1992,4.6
4,1993,2.6


In [8]:
# lowercase all
cpih.columns = [column.lower() for column in cpih.columns]

cpih_filt=cpih[cpih['year'].str.len() == 4]



In [9]:
cpih_filt['year']=cpih_filt['year'].astype(int)


/var/folders/_x/gqxjdk0n3k3b_ss6j7w47l780000gn/T/ipykernel_23240/44165346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpih_filt['year']=cpih_filt['year'].astype(int)


In [10]:
cpih_filt=cpih_filt[(cpih_filt['year']>2013) & (cpih_filt['year']<2023)]
cpih_filt
cpih_filt = cpih_filt.rename(columns={'cpih': 'rate'})


# Cleaning Bank Rate Data
For this dataset, the data is set by month, but not always updated each month. The data will be calculated by average by year.

In [11]:
bank_rate.columns = [column.lower() for column in bank_rate.columns]

bank_rate = bank_rate.rename(columns={'year': 'date'})
bank_rate = bank_rate.rename(columns={'bank_rate': 'rate'})
bank_rate.head()


,date,rate
0,01 Dec 75,11.50
1,01 Feb 88,8.88
2,01 Mar 76,9.25
3,01 Mar 79,13.00
4,01 Nov 82,9.38


In [13]:
# separate the year from the date.
bank_rate['year'] = bank_rate['date'].str[-2:]


In [14]:
# add '20' to the front of the year.
bank_rate['year'] = '20'+bank_rate['year']

bank_rate.head()

,date,rate,year
0,01 Dec 75,11.50,2075
1,01 Feb 88,8.88,2088
2,01 Mar 76,9.25,2076
3,01 Mar 79,13.00,2079
4,01 Nov 82,9.38,2082


In [15]:
# change 'year' datatype to int
bank_rate['year']=bank_rate['year'].astype(int)

In [16]:
# sort data from 2009
bank_rate=bank_rate[(bank_rate['year']>2008) & (bank_rate['year']<2023)]

In [18]:
# group data by year and show only 'year' and 'rate'
bank_rate_filt = bank_rate.groupby('year')['rate'].mean().reset_index()
bank_rate_filt
# cannot find 2014-2015 rate. 2010-2015 data unavailable!

,year,rate
0,2009,1.000
1,2016,0.250
2,2017,0.500
3,2018,0.750
4,2020,0.175
5,2021,0.250
6,2022,1.750


In [19]:
# here we change the 2009 to 2014 data manually, to match the other dataset. 
# The data is taken from the Bank of England's website. Which also takes the average of said year.
condition = bank_rate_filt['year'] == 2009

bank_rate_filt.loc[condition, 'year'] = 2014
condition = bank_rate_filt['year'] == 2014

# the data from Bank of England is input here.
bank_rate_filt.loc[condition, 'rate'] = 0.5

In [20]:
bank_rate_filt

,year,rate
0,2014,0.500
1,2016,0.250
2,2017,0.500
3,2018,0.750
4,2020,0.175
5,2021,0.250
6,2022,1.750


# last data clean: sort all data then merge

In [21]:
gdp_filt = gdp_filt.sort_values(by='year', ascending=True)
cpih = cpih.sort_values(by='year', ascending=True)
bank_rate_filt = bank_rate_filt.sort_values(by='year', ascending=True)


In [22]:
# add identification to each row in a new column 'type'
gdp_filt['type']='GDP'
cpih_filt['type']='Interest Rate (CPIH)'
bank_rate_filt['type']='Bank Rate (BoE)'

In [23]:
df=pd.concat([gdp_filt,cpih_filt,bank_rate_filt])

In [26]:
df.describe()['rate']

count    25.000000
mean      1.178000
std       1.664384
min      -1.200000
25%       0.400000
50%       0.675000
75%       1.700000
max       7.900000
Name: rate, dtype: float64

# Export Dataframe to Excel

In [ ]:
df.to_excel('finance_indicator.xlsx', index=False)

And that concludes the data cleaning for financial indicator.